In [1]:
import os
os.environ['CUDA_PATH']='/c/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v12.2'
import spacy
import regex as re
import pandas as pd

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
df = pd.read_csv('data/LeadSquared_assignment/data.csv')
df.drop(columns = ['Unnamed: 0', 'Age', 'Division Name', 'Department Name', 'Class Name'], inplace = True)
print(len(df))
df.drop_duplicates(subset = ['Review Text'], inplace=True)
df.reset_index(drop = True, inplace= True)
df

23486


,Clothing ID,Title,Review Text,Rating,Recommended IND,Positive Feedback Count
0,767,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0
1,1080,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4
2,1077,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0
3,1049,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0
4,847,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6
...,...,...,...,...,...,...
22630,1104,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0
22631,862,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0
22632,1104,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1
22633,1084,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2


In [49]:
subset = df[df['Clothing ID']==1078]

In [45]:
class TextAnalyzer:
    nlp = 
    def get_noun_subject(self, token, count=3):
        current_head = token.head
        while token != current_head:
            # print(current_head)
            if current_head.pos_ == 'NOUN':
                return current_head
            current_head = current_head.head
            count-=1
            if count==0:
                return token
        return token

    def get_verb_subject(self, doc, count=3):
        possible_subjects = []
        for word in doc:
            if word.pos_ == 'VERB':
                modifier = self.get_adj_from_parent(word)
                if modifier:
                    possible_subjects.append((modifier, word))
        
        return possible_subjects

    def get_adj_from_children(self, token, count=2):
        all_children = list(token.children)
        possible_adj = ''
        for child in all_children:
            if child.pos_ == 'NOUN' and possible_adj == '':
                possible_adj = self.get_adj_from_children(child)
            elif child.pos_ == 'ADJ':
                return child.text
        return possible_adj

    def get_adj_from_parent(self, token, count=2):
        head = token.head
        if head.pos_ == 'ADJ':
            return head.text
        all_children = list(head.children)
        for child in all_children:
            if child.pos_ == 'ADJ':
                return child.text
        if count-1==0:
            return ''
        return self.get_adj_from_parent(head, count=count-1)

    def extract_subject(self, text):
        doc = self.nlp(text)
        possible_subjects = []
        subject = ''
        subject_length = 0
        for token in doc:
            if token.pos_ == 'NOUN':
                # print('\n',token)
                head = self.get_noun_subject(token)
                modifier = self.get_adj_from_children(head)
                if not modifier:
                    modifier = self.get_adj_from_parent(head)
                start, end= (head.i, token.i+1) if head.i<=token.i else (token.i, head.i+1)
                possible_subjects.append((modifier, doc[start:end]))
                if end-start>subject_length:
                    subject_length = end-start
                    subject = possible_subjects[-1]
        
        if not subject:
            return self.get_verb_subject(doc)
        return subject


In [55]:
# for token in doc:
#     print(token.text,'\t' , token.pos_,'\t' , token.tag_,'\t' , token.dep_,'\t', token.sentiment, '\t' , token.i,'\t' ,token.head, '\t',list(token.children))

text = 'I really wanted this to work'
doc = nlp(text)
text_obj = TextAnalyzer()
op = text_obj.extract_subject(doc)
op

[]

In [56]:
# text = df.loc[0, 'Review Text']
# text = 'The phone is good value for money'
# print(text)
print(re.split(' *[.!?] *', text))
doc = nlp(text)
spacy.displacy.serve(doc, style="dep")

['I really wanted this to work']


d:\VS Code\lsassignment\lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [43]:
# op[0][1].text

for token in doc:
    print(token.text, '\t', token.pos_, '\t', token.tag_, '\t', token.dep_)

it 	 PRON 	 PRP 	 nsubj
is 	 AUX 	 VBZ 	 ROOT
worth 	 ADJ 	 JJ 	 acomp
buying 	 VERB 	 VBG 	 xcomp


In [53]:
data = subset

len(data)
split_data = pd.DataFrame()
for index, row in data.iterrows():
    row_data = pd.DataFrame(columns = ['review_id', 'clothing_id', 'title', 'text', 'rating', 'recommended', 'positive_feedback_count'])
    row_data['text'] = re.split(' *[.!?] *', row['Review Text'])
    row_data['review_id'] = index
    row_data['clothing_id'] = row['Clothing ID']
    row_data['title'] = row['Title']
    row_data['rating'] = row['Rating']
    row_data['recommended'] = row['Recommended IND']
    row_data['positive_feedback_count'] = row['Positive Feedback Count']

    split_data = pd.concat([split_data, row_data])


In [54]:
split_data['subject'] = split_data['text'].progress_apply(lambda x: )

,review_id,clothing_id,title,text,rating,recommended,positive_feedback_count
0,69,1078,Great summer fabric!,I really wanted this to work,3,0,1
1,69,1078,Great summer fabric!,"alas, it had a strange fit for me",3,0,1
2,69,1078,Great summer fabric!,"the straps would not stay up, and it had a wei...",3,0,1
3,69,1078,Great summer fabric!,"it worked standing up, but the minute i sat do...",3,0,1
4,69,1078,Great summer fabric!,the fabric was beautiful,3,0,1
...,...,...,...,...,...,...,...
0,22616,1078,Falls nicely,I bought this dress for work and post work hap...,5,1,0
1,22616,1078,Falls nicely,love it,5,1,0
2,22616,1078,Falls nicely,it's material is high quality and is flatterin...,5,1,0
3,22616,1078,Falls nicely,it's a little shorter than i thought but still...,5,1,0
